In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexSets

In [ ]:
gcs = GraphOfConvexSets()

rooms = [
([0, 5], [2, 8]),
([0, 3], [4, 5]),
([0, 0], [2, 3]),
([2, 5], [4, 8]),
([2, 0], [4, 3]),
([4, 0], [6, 8]),
([6, 7], [8, 8]),
([6, 1], [8, 7]),
([6, 0], [8, 1]),
([8, 0], [10, 8]),
([10, 5], [12, 8]),
([10, 4], [12, 5]),
([10, 1], [12, 4]),
([10, 0], [12, 1]),
([12, 0], [14, 8]),
([14, 3], [16, 8]),
([14, 0], [16, 3]),
]

def add_room(n, l, u):
    v = gcs.add_vertex(f"room{n}")
    x1 = v.add_variable(2)
    x2 = v.add_variable(2)
    v.add_constraint(x1 >= l)
    v.add_constraint(x2 >= l)
    v.add_constraint(x1 <= u)
    v.add_constraint(x2 <= u)
    v.add_cost(cp.norm(x2 - x1, 2))
    return v

for n, room in enumerate(rooms):
    add_room(n, *room)

In [ ]:
doors = [
(0, 3, [2, 7], [2, 8]),
(1, 2, [0, 3], [1, 3]),
(1, 5, [4, 4], [4, 5]),
(2, 4, [2, 2], [2, 3]),
(3, 5, [4, 5], [4, 8]),
(4, 5, [4, 0], [4, 3]),
(5, 6, [6, 7], [6, 8]),
(5, 7, [6, 3], [6, 4]),
(5, 8, [6, 0], [6, 1]),
(6, 9, [8, 7], [8, 8]),
(7, 9, [8, 3], [8, 4]),
(8, 9, [8, 0], [8, 1]),
(9, 10, [10, 7], [10, 8]),
(9, 11, [10, 4], [10, 5]),
(9, 13, [10, 0], [10, 1]),
(10, 14, [12, 5], [12, 6]),
(11, 14, [12, 4], [12, 5]),
(12, 14, [12, 1], [12, 2]),
(13, 14, [12, 0], [12, 1]),
(14, 15, [14, 7], [14, 8]),
(14, 16, [14, 2], [14, 3]),
]

def add_door(n, m, l, u):
    tail = gcs.vertices[n]
    head = gcs.vertices[m]
    e = gcs.add_edge(tail, head)
    e.add_constraint(tail.variables[1] == head.variables[0])
    e.add_constraint(tail.variables[1] >= l)
    e.add_constraint(tail.variables[1] <= u)
    return e

def add_two_doors(n, m, l, u):
    e1 = add_door(n, m, l, u)
    e2 = add_door(m, n, l, u)
    return e1, e2
    
for door in doors:
    add_two_doors(*door)

In [ ]:
s = gcs.get_vertex_by_name("room2")
t = gcs.get_vertex_by_name("room15")
prob = gcs.solve_shortest_path(s, t)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches


def plot_room(n, l, u):
    l = np.array(l)
    u = np.array(u)
    rect = patches.Rectangle(l, *(u - l), ec='k', fc='mintcream')
    c = (l + u) / 2
    plt.text(*c, n, zorder=3, ha='center', va='center')
    plt.gca().add_patch(rect)
    
def plot_door(l, u):
    l = np.array(l)
    u = np.array(u)
    rect = patches.Rectangle(l, *(u - l), ec='lightgreen', fc='lightgreen')
    plt.gca().add_patch(rect)
    
plt.figure()

for n, room in enumerate(rooms):
    plot_room(n, *room)
    
for door in doors:
    plot_door(door[2], door[3])

for vertex in gcs.vertices:
    if vertex.y.value > 0.5:
        x1, x2 = vertex.variables
        values = np.array([x1.value, x2.value]).T
        plt.plot(*values, c='b', linestyle='--')